In [1]:
import os, sys

colab = False
# #### COLAB ONLY CELL
if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    # drive.mount('/content/gdrive',force_remount=True)
    os.chdir("/content/gdrive/MyDrive/COMEXT")
    !pip install -q -r world-trade-thesis/requirements.txt
    # !pwd
    sys.path.append('/content/gdrive/MyDrive/COMEXT/world-trade-thesis/src')
else:
    os.chdir("H:/My Drive/COMEXT")
    print(os.getcwd())
    sys.path.append('H:/My Drive/COMEXT/world-trade-thesis/src')

import pandas as pd
import networkx as nx
import numpy as np
import graph_analysis as g
from tqdm.notebook import tqdm, trange

# exec(open("world-trade-thesis/src/graph_analysis.py","r").read())

df_pop, eu_iso = g.load_population_df()

H:\My Drive\COMEXT
Functions loaded!


In [ ]:
# This script is to run the SBM on all years, all products graphs
# !Rscript world-trade-thesis/src/complete_sbm.r

In [26]:
dfs_prod_sbm = {}
dfs_prod_dc = {}
df_mod_icl = pd.DataFrame(columns = ["year","prod","type","mod","icl"])
for f in tqdm(os.listdir("data-samples/sbm/v4/")):
    if "ini" not in f:
        df = pd.read_csv("data-samples/sbm/v4/"+f,na_filter=False)
        f_l = f.split("_")
        y = f_l[1][1:]
        p = f_l[2][1:3]
        # print(y,p)
        mod = df.modularity[0]
        icl = df.icl[0]
        if "sbm" in f:
            type = "sbm"
            if p not in dfs_prod_sbm:
                dfs_prod_sbm[p] = df.set_index("country")[["block"]].rename({"block":y},axis=1)
            else:
                dfs_prod_sbm[p] = pd.concat([dfs_prod_sbm[p], df.set_index("country")[["block"]].rename({"block":y},axis=1)],axis=1)
        elif "dc" in f:
            type = "dc"
            if p not in dfs_prod_dc:
                dfs_prod_dc[p] = df.set_index("country")[["block"]].rename({"block":y},axis=1)
            else:
                dfs_prod_dc[p] = pd.concat([dfs_prod_dc[p], df.set_index("country")[["block"]].rename({"block":y},axis=1)],axis=1)
        df_mod_icl = pd.concat([df_mod_icl,pd.DataFrame([[y,p,type,mod,icl]],columns=["year","prod","type","mod","icl"])],axis=0)
    df_mod_icl.reset_index(drop=True,inplace=True)
    # break

In [32]:
df.set_index("country")[["block"]].rename({"block":y},axis=1)
df.modularity[0]
df.icl[0]

-1061.16871298038

In [31]:
for p in tqdm(dfs_prod_sbm):
    dfs_prod_sbm[p].to_csv(f"data-samples/sbm_allyears/v4/complete_allyears_p{p}_sbm.csv")
    dfs_prod_dc[p].to_csv(f"data-samples/sbm_allyears/v4/complete_allyears_p{p}_dc.csv")

# dfs_prod = {}
# for f in tqdm(os.listdir("data-samples/sbm_allyears/v2/")):
#     if "ini" not in f:
#         p = f.replace(".csv","").split("_p")[-1]
#         dfs_prod[p] = pd.read_csv("data-samples/sbm_allyears/v2/"+f, index_col=0, na_filter=False)

In [ ]:
metrics = pd.read_parquet("data-samples/metrics/metrics_complete_V_2.parquet")

In [ ]:
# m2 = metrics.set_index(["country","prod","year"])
# m2["sbm_block"] = 0
# sbm = dfs_prod["30"].reset_index().rename({"30":"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")

In [ ]:
metrics = metrics.sort_values(["country","prod","year"]).set_index(["country","prod","year"])

In [ ]:
df_prods = None
for p in tqdm(dfs_prod):
    if df_prods is None:
        sbm = dfs_prod[p].copy()
        sbm[p] = p
        df_prods = sbm.reset_index().rename({p:"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")
        # break
    else:
        sbm = dfs_prod[p].copy()
        sbm[p] = p
        df_prods = pd.concat([df_prods,sbm.reset_index().rename({p:"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")])

In [ ]:
# a = set(metrics.loc[:,"01",:].index)
# b = set(df_prods.reset_index().set_index(["country","level_2"]).index)

In [ ]:
metrics["sbm_block"] = "0"
for c, p, y in tqdm(metrics.index):
    try:
        metrics.loc[(c,p,y),"sbm_block"] = str(df_prods.loc[c,p,y])
    except:
        print(c,p,y)

# sbms = df_prods.reset_index().rename({"level_2":"year"},axis=1).sort_values(["country","prod","year"]).reset_index(drop=True).set_index(["country","prod","year"]).values
# metrics["sbm_block"] = sbms

XK 01 2005
XK 01 2006
XK 01 2007
XK 01 2008
XK 01 2009
XK 01 2010
XK 01 2011
XK 01 2012
XK 01 2013
XK 01 2014
XK 01 2015
XK 01 2016
XK 01 2017
XK 01 2018
XK 01 2019
XK 01 2020
XK 02 2005
XK 02 2006
XK 02 2007
XK 02 2008
XK 02 2009
XK 02 2010
XK 02 2011
XK 02 2012
XK 02 2013
XK 02 2014
XK 02 2015
XK 02 2016
XK 02 2017
XK 02 2018
XK 02 2019
XK 02 2020
XK 03 2005
XK 03 2006
XK 03 2007
XK 03 2008
XK 03 2009
XK 03 2010
XK 03 2011
XK 03 2012
XK 03 2013
XK 03 2014
XK 03 2015
XK 03 2016
XK 03 2017
XK 03 2018
XK 03 2019
XK 03 2020
XK 05 2006
XK 05 2008
XK 05 2009
XK 05 2010
XK 05 2011
XK 05 2012
XK 05 2013
XK 05 2014
XK 05 2015
XK 05 2016
XK 05 2017
XK 05 2018
XK 05 2019
XK 05 2020
XK 06 2014
XK 06 2015
XK 06 2016
XK 06 2017
XK 06 2018
XK 06 2019
XK 06 2020
XK 07 2007
XK 07 2008
XK 07 2009
XK 07 2010
XK 07 2011
XK 07 2012
XK 07 2013
XK 07 2014
XK 07 2015
XK 07 2016
XK 07 2017
XK 07 2018
XK 07 2019
XK 07 2020
XK 08 2005
XK 08 2006
XK 08 2007
XK 08 2008
XK 08 2009
XK 08 2010
XK 08 2011
XK 08 2012

In [ ]:
metrics

month  edges  nodes          size  degree  degree_perc  \
country prod year                                                          
AD      01   2001     -   8522    214  9.070524e+06      16     0.075117   
             2002     -   8528    216  9.262277e+06      21     0.097674   
             2003     -   8774    216  9.144775e+06      20     0.093023   
             2004     -   8775    214  9.658745e+06      16     0.075117   
             2005     -   8722    216  9.547236e+06      21     0.097674   
...                 ...    ...    ...           ...     ...          ...   
ZW      XX   2018     -   5442    156  1.729591e+07      88     0.567742   
             2019     -   5230    156  1.921495e+07      77     0.496774   
             2020     -   2617    157  8.860344e+06      14     0.089744   
        YY   2019     -    621    191  3.015189e+05       2     0.010526   
             2020     -    641    191  4.845990e+05       3     0.015789   

                   out_degree  out_degree_perc  in_degree  in_degree_perc  \
country prod year                                                           
AD      01   2001          10         0.046948          6        0.028169   
             2002          11         0.051163         10        0.046512   
             2003          14         0.065116          6        0.027907   
             2004          10         0.046948          6        0.028169   
             2005          16         0.074419          5        0.023256   
...                       ...              ...        ...             ...   
ZW      XX   2018          28         0.180645         60        0.387097   
             2019          26         0.167742         51        0.329032   
             2020          14         0.089744          0        0.000000   
        YY   2019           1         0.005263          1        0.005263   
             2020           2         0.010526          1        0.005263   

                   ...  in_degree_xmin  out_degree_gamma  out_degree_xmin  \
country prod year  ...                                                      
AD      01   2001  ...             1.0          1.311003              1.0   
             2002  ...             1.0          1.313348              1.0   
             2003  ...             1.0          1.305121              1.0   
             2004  ...             1.0          1.307934              1.0   
             2005  ...             1.0          1.314439              1.0   
...                ...             ...               ...              ...   
ZW      XX   2018  ...             1.0          1.297004              1.0   
             2019  ...             1.0          1.300829              1.0   
             2020  ...             1.0          1.385909              1.0   
        YY   2019  ...             1.0          2.308747              1.0   
             2020  ...             1.0          2.160862              1.0   

                   in_weight_degree_gamma  in_weight_degree_xmin  \
country prod year                                                  
AD      01   2001                1.113281                    1.0   
             2002                1.114368                    1.0   
             2003                1.113026                    1.0   
             2004                1.113712                    1.0   
             2005                1.114848                    1.0   
...                                   ...                    ...   
ZW      XX   2018                1.087884                    1.0   
             2019                1.085950                    1.0   
             2020                1.085143                    1.0   
        YY   2019                1.286633                    1.0   
             2020                1.294406                    1.0   

                   out_weight_degree_gamma  out_weight_degree_xmin  \
country prod year                                                    


In [ ]:
metrics = metrics.reset_index()

In [ ]:
metrics.to_parquet("data-samples/metrics/metrics_complete_V_2_sbm2.parquet")